In [9]:
from pathlib import Path

def make_sub_str(model_path, sweep_par, val, exp_dir):
    
    base_str = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'   
    
    base_str = base_str + ' +experiment=N2_352_11'
    
    return base_str + ' ' + sweep_par + f'={val} run_name={exp_dir}/{sweep_par}:{val}' + '\n'

In [10]:
model_path = Path('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod')
exp_dir = 'N2_9'
# exp_dir = 'simfish2'

sweep_par = 'supervised.step_size'
sweep_vals = [1000,2000,5000,120000]

In [11]:
Path.mkdir(model_path/exp_dir, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    f.write(f'rm -r /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/fishcod/runs/{exp_dir} \n')
    for v in sweep_vals:  
        exec_str = make_sub_str(model_path, sweep_par, v, exp_dir)
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=N2_352_11 supervised.step_size=1000 run_name=N2_9/supervised.step_size:1000

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=N2_352_11 supervised.step_size=2000 run_name=N2_9/supervised.step_size:2000

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=N2_352_11 supervised.step_size=5000 run_name=N2_9/supervised.step_size:5000

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/